In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import shutil
import os

In [3]:
final_urls=[]

url_start='https://www.bild.bundesarchiv.de/dba/en/search/?yearfrom=&yearto=&query='
final_search= str(input("Insert term from whose search the results will be scraped: "))
search_term= final_search
url_end='&page='
num_page= int(input("Insert total number of pages for the search term: "))
num_page+=1
for search in range(1,num_page):
    final_search=str(url_start+final_search+url_end+str(search))
    reqs = requests.get(final_search)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    txt= '(?:(?:https?):\/\/)?[\w/\-?=%.]+\.jpg'
    for url in re.findall(txt,str(soup)):
        final_urls.append(url)

print("Image URLs extracted...")
final_df= pd.DataFrame()
final_df['site_images']= final_urls
final_df= final_df[final_df['site_images'].str.contains('https://bild.bundesarchiv.de/device_barch')==True]
final_df= final_df.drop_duplicates()
csv_name= search_term+"_image_urls"+".csv"
final_df.to_csv(csv_name,index=False)
final_urls= final_df['site_images'].to_list()

search_term= search_term.replace(' ','')
search_term=search_term.replace(',','')
os.mkdir(os.path.join(os.getcwd(),"images_from_the_past",'Bundesarchiv',search_term))

for url in final_urls:
    try:
        image_url = url
        filename = image_url.split("/")[-1]
        filename=filename.replace("/","_")
        filename= os.path.join(os.getcwd(),"images_from_the_past",'Bundesarchiv',search_term,filename)
        r = requests.get(image_url, stream = True)

        # Check if the image was retrieved successfully
        if r.status_code == 200:
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            # Open a local file with wb ( write binary ) permission.
            with open(filename,'wb') as f:
                shutil.copyfileobj(r.raw, f)
        else:
            print('Image Couldn\'t be retreived')
    except:
        continue

Insert term from whose search the results will be scraped: Nationalsozialistische
Insert total number of pages for the search term: 171
Image URLs extracted...
